# Timegan_tsgm

Requirements:

pip install tsgm

https://tsgm.readthedocs.io/en/latest/

In [1]:
import tsgm
import numpy as np
import pandas as pd

In [16]:
# Timegan_tsgm precisa de dados sequenciados

data_path = "../../data/sequences/data_train_act_13_windows.npy"

data = np.load(data_path, allow_pickle=True)

data.shape

(44787, 600, 6)

In [ ]:
from tsgm.models.timeGAN import TimeGAN
import tensorflow as tf

timegan = TimeGAN(
    seq_len=data.shape[1],        # Length of the time series
    module='gru',                    # Recurrent module type (gru, lstm, or lstmLN)
    hidden_dim=24,                    # Hidden layer dimension
    n_features=data.shape[2],   # Number of features in your data
    n_layers=3,                  # Number of recurrent layers
    batch_size=256,                  # Batch size for training
    gamma=1.0                     # Hyperparameter for adversarial loss
)

# legacy é só para arm macs, fora isso usar o atual.
timegan.compile(
    d_optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=1e-4),
    g_optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=1e-4),
    emb_optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=1e-4),
    supgan_optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=1e-4),
    ae_optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=1e-4),
    emb_loss=tf.keras.losses.MeanSquaredError(),
    clf_loss=tf.keras.losses.BinaryCrossentropy()
)



In [19]:
timegan.fit(
    data=data,                 
    epochs=100,             
    checkpoints_interval=10, 
)

Model: "Autoencoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 RealData (InputLayer)       [(256, 600, 6)]           0         
                                                                 
 Embedder (Sequential)       (256, 600, 24)            10104     
                                                                 
 Recovery (Sequential)       (256, 600, 6)             10950     
                                                                 
Total params: 21054 (82.24 KB)
Trainable params: 21054 (82.24 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Model: "AdversarialSupervised"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 RandomNoise (InputLayer)    [(256, 600, 6)]           0         
                                                      

INFO:models:Start Embedding Network Training
Autoencoder - training: 100%|██████████| 100/100 [01:48<00:00,  1.08s/it]
INFO:models:Finished Embedding Network Training
INFO:models:Start Training with Supervised Loss Only
Adversarial Supervised - training: 100%|██████████| 100/100 [01:21<00:00,  1.23it/s]
INFO:models:Finished Training with Supervised Loss Only
INFO:models:Start Joint Training
GAN with embedding - training:   0%|          | 0/100 [00:00<?, ?it/s]INFO:models:Train Discriminator (discriminator does not work well yet)
INFO:models:step: 0/100,
                    d_loss: 2.0553998947143555,
                    g_loss_u: 0.7056000232696533,
                    g_loss_u_e: 0.7085000276565552,
                    g_loss_s: 0.060499999672174454,
                    g_loss_v: 5.105500221252441,
                    g_loss_v: 518.0195922851562,
                    e_loss_t0: 4.574100017547607
GAN with embedding - training:  10%|█         | 10/100 [04:05<34:37, 23.09s/it]INFO:models:

In [24]:
synthetic_data = timegan.generate(n_samples=100000)

Synthetic data generation: 100%|██████████| 391/391 [33:39<00:00,  5.17s/it] 


In [25]:
synthetic_data.shape

(100000, 600, 6)

In [26]:
np.save("timegan_tsgm_100epochs_synthetic_data_act_13_windows.npy", synthetic_data)